In [1]:
import pickle

import altair as alt

import pandas as pd

import polyclonal

import warnings
warnings.filterwarnings('ignore')

from IPython.utils import io

In [2]:
import os
os.chdir('../../')

### Get mut escape df's for each serum, adding age cohort

In [3]:
adult_sera = ['33C', '34C', '197C', '199C', '215C', '210C', '74C', '68C', '150C', '18C']
teen_sera = [2343, 2350, 2365, 2380, 2382, 3866, 3856, 3857, 3862, 3895]
ped_sera = [2367, 3944, 2462, 2389, 2323, 2388, 2463, 3973, 4299, 4584]

serum_lists = [ped_sera, teen_sera, adult_sera]
age_cohorts = ['0-5', '15-18', '40-45']

df_list = []

i = 0
for list in serum_lists:
    for serum in list:
        model = pickle.load(open(f'results/antibody_escape/{serum}.pickle', 'rb'))
        df = model.mut_escape_df
        df['serum'] = serum
        df['age_cohort'] = age_cohorts[i]

        df_list.append(df)

    i+=1

In [4]:
def plot_sitewise_aa_scores(mut_escape_df_list, site):
    site_filtered_dfs = []
    for df in mut_escape_df_list:
        df_filt = df.loc[df['site'] == site]
        site_filtered_dfs.append(df_filt)

    sitewise_aa_df = pd.concat(site_filtered_dfs).reset_index(drop=True)

    sitewise_plot = (
        alt.Chart()
        .encode(
            x=alt.X("mutant", 
                    title="mutant",
                   ),
            y=alt.Y(
                "escape_mean",
                title="mean escape",
            ),
            # xOffset='jitter:Q',
            color=alt.Color('serum:N'),
                           # ).scale(scheme='set2'),
            # detail='serum',
        # ).transform_calculate(jitter="sqrt(-2*log(random()))*cos(2*PI*random())")
        ).mark_circle(size=50, opacity=0.7)
        .properties(width=400, height=200)
        # .mark_circle(opacity=0.7, size=20))
    )

    x_axis = alt.Chart(pd.DataFrame({'y': [0]})).mark_rule(opacity=0.5).encode(y='y')
    
    chart = alt.layer(
        sitewise_plot, x_axis, data=sitewise_aa_df
    ).facet(
        facet=alt.Facet(
            'age_cohort:N',
            title=f'AA-level escape at site {site}',
            header=alt.Header(
                titleFontSize=23,
                # titleFontWeight='normal',
                labelFontSize=17,
            )
        ),
        columns=2
    ).configure_axis(
        grid=False,
        labelFontSize=13,
        titleFontSize=15
    ).resolve_axis(
        x='independent'
    )
    
    return chart

In [5]:
plot_sitewise_aa_scores(df_list, '145')

alt.FacetChart(...)